In [ ]:
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
# Load YOLOv4 model
class_names = [c.strip() for c in open('coco.names').readlines()]
model = tf.saved_model.load('./checkpoints/yolov4-416')

In [ ]:
def load_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (416, 416))
    img = img / 255.0
    img = img[np.newaxis, ...].astype(np.float32)
    return img

def detect_objects(image_path):
    img = load_image(image_path)
    infer = model.signatures['serving_default']
    batch_data = tf.constant(img)
    pred_bbox = infer(batch_data)
    for key, value in pred_bbox.items():
        boxes = value[:, :, 0:4]
        pred_conf = value[:, :, 4:]

    return boxes, pred_conf

def draw_bbox(image_path, boxes, scores, classes, class_names):
    image = cv2.imread(image_path)
    image_h, image_w, _ = image.shape
    for i in range(len(boxes)):
        if scores[i] > 0.5:
            box = boxes[i]
            class_id = int(classes[i])
            score = scores[i]
            x_min, y_min, x_max, y_max = box
            x_min = int(x_min * image_w)
            y_min = int(y_min * image_h)
            x_max = int(x_max * image_w)
            y_max = int(y_max * image_h)
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            label = f'{class_names[class_id]}: {score:.2f}'
            cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imshow('Detected Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
 # Path to the input image
image_path = 'path/to/your/image.jpg'

# Detect objects
boxes, scores, classes = detect_objects(image_path)

# Draw bounding boxes on the image
draw_bbox(image_path, boxes[0], scores[0], classes[0], class_names)